In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split as splt
# for handling the outliers
from sklearn.ensemble import IsolationForest
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor
from tensorflow.keras.optimizers import Adadelta
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import load_model
from tensorflow.keras.models import save_model

## 1 IMPORTING THE DATA

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [29]:
print(train.head())
print(train.tail())

        id  week  checkout_price  base_price  num_orders  101  102  104  106  \
0  1379560     1          136.83      152.29         177    0    0    0    0   
1  1466964     1          136.83      135.83         270    0    0    0    0   
2  1346989     1          134.86      135.86         189    0    0    0    0   
3  1338232     1          339.50      437.53          54    0    0    0    0   
4  1448490     1          243.50      242.50          40    0    0    0    0   

   108  ...  2640  2664  2704  2707  2760  2826  2867  2956  1  1  
0    0  ...     0     0     0     0     0     0     0     0  0  0  
1    0  ...     0     0     0     0     0     0     0     0  0  0  
2    0  ...     0     0     0     0     0     0     0     0  0  0  
3    0  ...     0     0     0     0     0     0     0     0  0  0  
4    0  ...     0     0     0     0     0     0     0     0  0  0  

[5 rows x 133 columns]
             id  week  checkout_price  base_price  num_orders  101  102  104  \
456543 

In [30]:
print(test.head())
print(test.tail())

        id  week  checkout_price  base_price  101  102  104  106  108  109  \
0  1028232   146          158.11      159.11    0    0    0    0    0    0   
1  1127204   146          160.11      159.11    0    0    0    0    0    0   
2  1212707   146          157.14      159.14    0    0    0    0    0    0   
3  1082698   146          162.02      162.02    0    0    0    0    0    0   
4  1400926   146          163.93      163.93    0    0    0    0    0    0   

   ...  2640  2664  2704  2707  2760  2826  2867  2956  1  1  
0  ...     0     0     0     0     0     0     0     0  0  0  
1  ...     0     0     0     0     0     0     0     0  0  0  
2  ...     0     0     0     0     0     0     0     0  0  0  
3  ...     0     0     0     0     0     0     0     0  0  0  
4  ...     0     0     0     0     0     0     0     0  0  0  

[5 rows x 132 columns]
            id  week  checkout_price  base_price  101  102  104  106  108  \
32568  1250239   155          482.09      484.09    

## 2 FEATURE ENGINEERING
- **2.1. CHANGING THE DATA TYPE**

In [5]:
X = train.drop('id,week,num_orders,checkout_price,base_price'.split(','),axis=1)
Y = test.drop('id,week,checkout_price,base_price'.split(','),axis=1)

In [6]:
X = X.astype(str)
Y = Y.astype(str)

In [26]:
print(X.describe())
print(Y.describe())

       checkout_price     base_price            101            102  \
count   423603.000000  423603.000000  423603.000000  423603.000000   
mean       340.704680     361.137211       0.013765       0.011317   
std        153.812665     161.269561       0.116515       0.105779   
min          2.970000      55.350000       0.000000       0.000000   
25%        241.530000     244.500000       0.000000       0.000000   
50%        309.430000     317.220000       0.000000       0.000000   
75%        446.230000     470.510000       0.000000       0.000000   
max        767.330000     767.330000       1.000000       1.000000   

                 104            106            108           109  \
count  423603.000000  423603.000000  423603.000000  423603.00000   
mean        0.013593       0.012944       0.014211       0.01368   
std         0.115794       0.113032       0.118362       0.11616   
min         0.000000       0.000000       0.000000       0.00000   
25%         0.000000       0.

In [8]:
train['center_id,meal_id,emailer_for_promotion,homepage_featured'.split(',')] = X
test['center_id,meal_id,emailer_for_promotion,homepage_featured'.split(',')] = Y

In [27]:
train.describe(include='all')

,id,week,checkout_price,base_price,num_orders,101,102,104,106,108,...,2640,2664,2704,2707,2760,2826,2867,2956,1,1
count,4.236030e+05,423603.000000,423603.000000,423603.000000,423603.000000,423603.000000,423603.000000,423603.000000,423603.000000,423603.000000,...,423603.000000,423603.000000,423603.000000,423603.000000,423603.000000,423603.000000,423603.000000,423603.000000,423603.000000,423603.000000
mean,1.250083e+06,74.904432,340.704680,361.137211,182.205542,0.013765,0.011317,0.013593,0.012944,0.014211,...,0.025082,0.021400,0.022986,0.019584,0.023878,0.022245,0.019027,0.007830,0.065347,0.087842
std,1.443480e+05,41.587735,153.812665,161.269561,169.519667,0.116515,0.105779,0.115794,0.113032,0.118362,...,0.156376,0.144713,0.149859,0.138567,0.152671,0.147479,0.136621,0.088143,0.247137,0.283065
min,1.000000e+06,1.000000,2.970000,55.350000,13.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.125012e+06,39.000000,241.530000,244.500000,53.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.250096e+06,76.000000,309.430000,317.220000,122.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.375126e+06,111.000000,446.230000,470.510000,270.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.499999e+06,145.000000,767.330000,767.330000,729.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [28]:
test.describe(include='all')

,id,week,checkout_price,base_price,101,102,104,106,108,109,...,2640,2664,2704,2707,2760,2826,2867,2956,1,1
count,3.257300e+04,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000,...,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000,32573.000000
mean,1.248476e+06,150.477819,341.854440,356.493615,0.013140,0.010438,0.014491,0.012587,0.014951,0.013447,...,0.022565,0.023639,0.022227,0.023455,0.021859,0.023639,0.016179,0.013078,0.066435,0.081356
std,1.441580e+05,2.864072,153.893886,155.150101,0.113875,0.101634,0.119503,0.111486,0.121359,0.115179,...,0.148513,0.151925,0.147423,0.151346,0.146224,0.151925,0.126166,0.113612,0.249045,0.273385
min,1.000085e+06,146.000000,67.900000,89.240000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.123969e+06,148.000000,214.430000,243.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.247296e+06,150.000000,320.130000,321.130000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.372971e+06,153.000000,446.230000,455.930000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.499996e+06,155.000000,1113.620000,1112.620000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


- **REMOVING OUTLIERS**

In [11]:
def outlier(data):
    qmax = {}
    qmin = {}
    for i in list(data.columns):
        if(np.issubdtype(data[i].dtype, np.number) == True):
            iqr = data[i].quantile(0.75) - data[i].quantile(0.25)
            qmax[i] = data[i].quantile(0.75) + 1.5*iqr
            qmin[i] = data[i].quantile(0.25) - 1.5*iqr

    for i in list(data.columns):
        if(np.issubdtype(data[i].dtype, np.number) == True):
            data = data.drop(data[(data[i] > qmax[i]) | (data[i] < qmin[i])].index)
    return data

In [12]:
train = outlier(train)
train.shape

(423603, 9)

- **2.2 CREATING DUMMY VARIABLES**

In [13]:
dataset = train
for i in 'center_id,meal_id,emailer_for_promotion,homepage_featured'.split(','):
    data = pd.get_dummies(dataset[i],drop_first=True)
    dataset = pd.concat([dataset,data],axis=1)
train = dataset
train.drop('center_id,meal_id,emailer_for_promotion,homepage_featured'.split(','),axis=1,inplace=True)

In [14]:
dataset = test
for i in 'center_id,meal_id,emailer_for_promotion,homepage_featured'.split(','):
    data = pd.get_dummies(dataset[i],drop_first=True)
    dataset = pd.concat([dataset,data],axis=1)
test = dataset
test.drop('center_id,meal_id,emailer_for_promotion,homepage_featured'.split(','),axis=1,inplace=True)

In [24]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 423603 entries, 0 to 456547
Columns: 133 entries, id to 1
dtypes: float64(2), int64(3), uint8(128)
memory usage: 71.1 MB


In [25]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32573 entries, 0 to 32572
Columns: 132 entries, id to 1
dtypes: float64(2), int64(2), uint8(128)
memory usage: 5.0 MB


In [17]:
new_train = train.drop(['id','week'],axis=1)

## 3 SEPARATING THE INDEPENDENT AND DEPENDENT VARIABLES

In [18]:
X = new_train.drop('num_orders',axis=1)
Y = new_train['num_orders']

In [19]:
# X.head()

In [20]:
new_test = test.drop(['id','week'],axis=1)

In [21]:
# new_test.head()

## 4 FEATURE SCALING

In [22]:
scale = StandardScaler()

In [23]:
data = X
# fitting and transforming the X into feature scaling
X = scale.fit_transform(X)
# converting again into dataframe
X = pd.DataFrame(X,columns=data.columns)
data = new_test
# fitting and transforming the test into feature scaling
new_test = scale.transform(new_test)
# converting again into dataframe
new_test = pd.DataFrame(new_test,columns=data.columns)

MemoryError: Unable to allocate array with shape (130, 423603) and data type float64

In [ ]:
X.shape

In [ ]:
new_test.shape

In [ ]:
print(X.head())
print(new_test.head())

## 5 SPLITTING THE X DATA INTO TRAIN AND TEST

In [ ]:
X_train, X_test, Y_train, Y_test = splt(X, Y, test_size=0.9781, random_state=42)

In [ ]:
X_train.info()

## 6 SAMPLE MODEL

In [31]:
def build_regressor():
    regressor = Sequential()
    regressor.add(Dense(units=66,kernel_initializer='normal',
                     activation='relu',input_dim=130))
    regressor.add(Dense(units=66,kernel_initializer='normal',
                     activation='relu'))
    regressor.add(Dense(units=66,kernel_initializer='normal',
                     activation='relu'))
    regressor.add(Dense(units=66,kernel_initializer='normal',
                     activation='relu'))
    regressor.add(Dense(units=66,kernel_initializer='normal',
                     activation='relu'))
    regressor.add(Dense(units=1,kernel_initializer='normal',
                     activation='relu'))
    regressor.compile(optimizer=Adadelta(),loss='mean_squared_logarithmic_error')
    return regressor
regressor = KerasRegressor(build_fn= build_regressor,batch_size=10,epochs=150)

In [32]:
regressor.fit(x=X,y=Y)

Train on 423603 samples
Epoch 1/150
423603/423603 [==============================] - 108s 256us/sample - loss: 2.9836
Epoch 2/150
423603/423603 [==============================] - 108s 254us/sample - loss: 0.9957
Epoch 3/150
423603/423603 [==============================] - 108s 255us/sample - loss: 0.8897
Epoch 4/150
423603/423603 [==============================] - 108s 254us/sample - loss: 0.7882
Epoch 5/150
423603/423603 [==============================] - 109s 258us/sample - loss: 0.6921
Epoch 6/150
423603/423603 [==============================] - 108s 256us/sample - loss: 0.6137
Epoch 7/150
423603/423603 [==============================] - 108s 256us/sample - loss: 0.5601
Epoch 8/150
423603/423603 [==============================] - 107s 252us/sample - loss: 0.5233
Epoch 9/150
423603/423603 [==============================] - 112s 265us/sample - loss: 0.4976
Epoch 10/150
423603/423603 [==============================] - 107s 253us/sample - loss: 0.4796
Epoch 11/150
423603/423603 [=======

In [33]:
regressor.model.save('model3.h5')

In [ ]:
# regressor.fit(x=X_train,y=Y_train)

In [ ]:
# parameters = {'batch_size': [10,100],
#              'epochs': [100,500]}

In [ ]:
# grid_search = GridSearchCV(estimator=regressor,
#                           param_grid=parameters,
#                           cv=8)

In [ ]:
# grid_search = grid_search.fit(X=X_train,y=Y_train)

In [ ]:
# batch=100,epochs=200,genpact score=58
msle = cross_val_score(estimator=regressor, X=X_train, y=Y_train, cv=8, n_jobs= -1)

In [ ]:
msle

In [ ]:
# batch=10,epochs=100,genpact score=
msle2 = cross_val_score(estimator=regressor, X=X_train, y=Y_train, cv=8, n_jobs= -1)

In [ ]:
msle2

### With Week And Different Tuning get score 103

In [ ]:
Y_pred = regressor.predict(new_test)

In [ ]:
Y_pred

In [ ]:
# id,num_orders

In [ ]:
submission = pd.DataFrame(Y_pred,columns=['num_orders'])

In [ ]:
submission = pd.concat([test['id'],submission],axis=1)

In [ ]:
submission.info()

In [ ]:
submission.head()

In [ ]:
test[['id']].head()

In [ ]:
test.shape

In [ ]:
submission.shape

In [ ]:
submission.to_csv('submission.csv',index=False)

### New Predition (Y_pred2)

In [ ]:
Y_pred2 = regressor.predict(new_test)

In [ ]:
submission2 = pd.DataFrame(Y_pred2,columns=['num_orders'])

In [ ]:
submission2 = pd.concat([test['id'],submission2],axis=1)

In [ ]:
submission2.describe()

In [ ]:
submission2.head()

In [ ]:
submission2.to_csv('submission2.csv',index=False)

## How To Save And Reload Model

- **SAVING THE MODEL**

In [ ]:
# My model name is regressor and I am using model.save function to the model
# in HDF5 format.
regressor.model.save('model.h5')

- **RELOADING THE MODEL**

In [ ]:
# Reassigning the model after loading the model
model = load_model('model.h5')

In [ ]:
# with batch=10, epochs=100
regressor.model.save('model2.h5')

In [ ]:
Y_pred = regressor.predict(new_test)

In [ ]:
Y_pred

In [ ]:
model2 = load_model('model2.h5')

In [ ]:
Y_pred2 = model2.predict(new_test)

In [ ]:
Y_pred2